### Client Setup

Accepting streaming data from server. Then training ensemble model for following ML models:
- LR
- SVM
- RF
- LDA
- KNN
- CART
- Naive-Bayes

<br>
<br>

In [6]:
# Importing required modules

# data streaming
import websockets

# data processing
from datetime import datetime
import pandas as pd

# ensemble modelling
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
# from tensorflow import keras
# from keras.models import load_model
# from keras.wrappers.scikit_learn import KerasClassifier

In [7]:
# Loading prebuilt model structures stored as '[MODEL].h5'

# # using keras.model.load_model and setting estimator type to 'classifier'
# model1 = load_model('../models/h5s/LSTM.h5')
# model1 = KerasClassifier(build_fn=model1)
# model1._estimator_type = "classifier"
# model2 = load_model('../models/h5s/GRU.h5')
# model2 = KerasClassifier(build_fn=model2)
# model2._estimator_type = "classifier"

# # using pickle.load
# model1 = pickle.load(open('../models/h5s/LSTM.h5', 'rb'))
# model1._estimator_type = "classifier"
# model2 = pickle.load(open('../models/h5s/GRU.h5', 'rb'))
# model2._estimator_type = "classifier"
model3 = pickle.load(open('../models/h5s/random-forest.h5', 'rb'))
model4 = pickle.load(open('../models/h5s/logistic-regression.h5', 'rb'))
model5 = pickle.load(open('../models/h5s/linear-discriminant-analysis.h5', 'rb'))
model6 = pickle.load(open('../models/h5s/kNN.h5', 'rb'))
model7 = pickle.load(open('../models/h5s/CART.h5', 'rb'))
model8 = pickle.load(open('../models/h5s/support-vector-machine.h5', 'rb'))
model9 = pickle.load(open('../models/h5s/naive-bayes.h5', 'rb'))

In [8]:
# Function to log specified content in specified file

def log(content="", new=False, file='stream.log', timestamp=False, print_line=0):
    """
    Logs specified content too specified pre-existing file

    :param content: describe about parameter p1
    :param new: if True then overwrites, else appends. Default is False
    :param file: destination file to save logs into
    :param timestamp: if True, adds a timestamp before the content and appends to a new line
    :param print_line: prints a line built with specified number of '-'
    :return: None
    """ 
    # to overwrite
    if(new):
        log = open(f'./{file}', 'w')
        log.write("")
        log.close()
    log = open(f'./{file}', 'a')

    # to add time stamp
    if(timestamp):
        log.write(f"\n{datetime.now()} ~ {content}")
    # without timestamp and new lines
    else:
        log.write(content)
    
    # to print a line
    if(print_line):
        line = "-" * print_line
        log.write(f"\n{line}\n\n")
    
    # save files with changes
    log.close()


In [9]:
async def ensemble(IoT, chunk_count, initial=False):
    """
    Logs specified content too specifie#d pre-existing file

    :param content: describe about parameter p1
    :param new: if True then overwrites, else appends. Default is False
    :param file: destination file to save logs into
    :param timestamp: if True, adds a timestamp before the content and appends to a new line
    :param print_line: prints a line built with specified number of '-'
    :return: None
    """ 
    # data preparation
    x = IoT.drop(['label', 'date', 'time'], axis=1)
    y = IoT['label']

    # data split into test and train sets
    x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2)
 
    # ensemble model
    # voting = VotingClassifier(estimators=[('SVM', model8), ('LR', model4), ('LSTM', model1), ('GRU', model2), ('RF', model3), ('LDA', model5), ('KNN', model6), ('CART', model7), ('Naive-Bayes', model9)], voting='hard')
    voting = VotingClassifier(estimators=[('SVM', model8), ('LR', model4), ('RF', model3), ('LDA', model5), ('KNN', model6), ('CART', model7), ('Naive-Bayes', model9)], voting='hard')
    voting.fit(x_train, y_train) 
    if(not initial):
        log(f"", file="ensemble.log", print_line=20)

    # log model version
    log(f"Model v{chunk_count}\n", initial, file="ensemble.log", timestamp=True)

    # log accuracy of each model
    # for clf in (model8, model4, model1, model2, model3, model5, model6, model7, model9, voting):
    for clf in (model8, model4, model3, model5, model6, model7, model9, voting):
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)

        # ---------------------------------------------------
        # SAVING INDIVIDUAL MODELS FOR BAYESIAN MODEL TRAINING
        # filename = f'./models/{clf.__class__.__name__}.h5'
        # pickle.dump(ensemble_model, open(filename, 'wb'))
        # ---------------------------------------------------

        log(f"{clf.__class__.__name__}: {accuracy_score(y_test, y_pred)}\n", file="ensemble.log")

    return voting

In [10]:
# define data chunk parameters
INITIAL_CHUNK_SIZE = 50   # size of chunk at start of data streaming
FINAL_CHUNK_SIZE = 20    # chunk size during data streaming
CHUNK_SIZE_INCREMENT_FACTOR = 2    # chunk size increment factor when error occurs while training ensemble model

async def client():
    """
    Client function for streaming and processing data simultaneously
    """

    uri = "ws://localhost:8765"    # websocket endpoint
    temp_row_count = 0    # realtime chunk size
    chunk_size = INITIAL_CHUNK_SIZE
    chunk_count = 0    # realtime chunk count
    initial_chunk = True    # initial chunk flag 
    model = 0    # init model
    log("", True)    # init 'stream.log' file
    async with websockets.connect(uri, ping_interval=None) as websocket:
        df = pd.DataFrame()    # init dataframe for storing the chunk

        # infinite loop for receiving streamed data
        while(True):
            # wait to receiving a record of data
            row = await websocket.recv()
            log(f"{temp_row_count} ")    # log row count
            data_list = row.split(",")    # split the row(comma separated format) string into a list 

            # if starting a new chunk, reinitialize 'df' to empty dataframe
            if(temp_row_count == 0):
                df = pd.DataFrame()
                # add row into the dataframe
                df = pd.DataFrame([data_list], columns = ["date", "time", "motion_status", "light_status", "label", "type"])
            
            #  else old chunk
            else:
                # append to 'df'
                df = pd.concat([df, pd.DataFrame([data_list], columns = ["date", "time", "motion_status", "light_status", "label", "type"])])
            temp_row_count += 1    # increment chunk size
            
            # 'tem_row_count' is equal to desired 'chunk_size'
            if (temp_row_count == chunk_size):
                # if initial chunk
                if(initial_chunk):
                    try:
                        # train ensemble model only once
                        model = await ensemble(df, chunk_count+1, initial=True)
                        # CHUNK ACCEPTED BY MODEL, NO ERRORS
                        initial_chunk = False    # initial chunk accepted, hence set to False
                        chunk_count += 1    # increment 'chunk_count'

                        # log chunk created
                        log(f"Chunk-{chunk_count} created with {temp_row_count} records.", timestamp=True, print_line=50)

                        chunk_size = FINAL_CHUNK_SIZE    # new chunk size reset to desired chunk size
                        temp_row_count = 0    # reset 'temp_row_count' for new chunk

                    # chunk rejected by ensemble model trainer, hence raise error
                    except ValueError:
                        # print(sys.exc_info())    # prints error statement/message

                        # log for class error
                        log("CLASS ERROR\n", timestamp=True)
                        chunk_size *= CHUNK_SIZE_INCREMENT_FACTOR    # multiplicative increase of chunk size
                
                # not initial chunk
                else:
                    # train ensemble model twice
                    try:
                        # 1. train ensemble model for chunk
                        new_chunk_model = await ensemble(df, chunk_count+1)
                        # 2. ensemble model of previous chunk('model') with model of new chunk('new_chunk_model)
                        new_ensemble_model = VotingClassifier(estimators=[('New Model', new_chunk_model), ('Old Model', model)], voting='hard')
                        
                        # ---------------------------------------------------
                        # SAVING MODEL FOR BAYESIAN MODEL TRAINING
                        filename = '../models/ensemble_model.h5'
                        pickle.dump(new_ensemble_model, open(filename, 'wb'))
                        # ---------------------------------------------------

                        model = new_ensemble_model    # set 'model' to new_ensemble_model'
                        chunk_count += 1    # increment 'chunk_count'

                        # log chunk created
                        log(f"Chunk-{chunk_count} created with {temp_row_count} records.", timestamp=True, print_line=50)
                        chunk_size = FINAL_CHUNK_SIZE    # new chunk size reset to desired chunk size
                        temp_row_count = 0    # reset 'temp_row_count' for new chunk

                    # chunk rejected by ensemble model trainer, hence raise error
                    except ValueError:
                        # print(sys.exc_info())    # prints error statement/message

                        # log for class error
                        log("CLASS ERROR\n", timestamp=True)
                        chunk_size *= CHUNK_SIZE_INCREMENT_FACTOR    # multiplicative increase of chunk size

            # send acknowledgement for receiving a row successfully
            await websocket.send("1")

await client()

CancelledError: 

<br>
<br>
<center><b>End of File</b></center>